In [1]:
class ArgMap(object):
    """
    Map storing the indices of input arguments.
    """

    def __init__(self, arg_map=[]):
        """
        ArgMap(arg_map=["A", "b", "x", ...]) set up argument map where each
        argument named in the `arg_map` list corresponds to its index.
        """
        object.__setattr__(self, "__map", dict())
        for i, am in enumerate(arg_map):
            setattr(self, am, i)


    def __getattr__(self, attr):
        return self.vmap()[attr]


    def __setattr__(self, attr, val):
        self.vmap()[attr] = val


    def __hasattr__(self, attr):
        return attr in self.vmap()


    def vmap(self):
        return object.__getattribute__(self, "__map")

In [2]:
am = ArgMap()

In [3]:
am.A = 1

In [4]:
am.A

1

In [5]:
am.B

KeyError: 'B'

In [6]:
am = ArgMap(["a", "B"])

In [7]:
am.a

0

In [8]:
am.B

1

In [9]:
from inspect import signature, getfullargspec

In [10]:
def f(x, y, a=1):
    return a

In [11]:
argspec = getfullargspec(f)
argspec

FullArgSpec(args=['x', 'y', 'a'], varargs=None, varkw=None, defaults=(1,), kwonlyargs=[], kwonlydefaults=None, annotations={})

In [12]:
def g(x, y, *args, a=1, **kwargs):
    return a+y

In [13]:
argspec = getfullargspec(g)
argspec

FullArgSpec(args=['x', 'y'], varargs='args', varkw='kwargs', defaults=None, kwonlyargs=['a'], kwonlydefaults={'a': 1}, annotations={})

In [14]:
a = [1,2,3,4,5,6]
for i in range(2, len(a)):
    print(i, a[i])

2 3
3 4
4 5
5 6


In [15]:
class ArgsView(object):

    def __init__(self, spec, args, kwargs):
        self.__spec = spec
        self.__args = args
        self.__kwargs = kwargs

        for i, arg_name in enumerate(self.__spec.args):
            setattr(
                self,
                arg_name,
                self.__arg_view(i, arg_name)
            )

        self._vargs = []
        for j in range(i+1, len(self.__args)):
            self._vargs.append(
                self.__varg_view(j)
            )

        for arg_name in self.__spec.kwonlyargs:
            setattr(
                self,
                arg_name,
                self.__kwoarg_view(arg_name)
            )


    @property
    def _spec(self):
        return self.__spec
    

    def _replace(self, name, data):
        setattr(self, name, lambda: data)


    def __arg_view(self, arg_idx, arg_name):
        if arg_idx < len(self.__args):
            return lambda: self.__args[arg_idx]
        if arg_name in self.__kwargs:
            return lambda: self.__kwargs[arg_name]

        defaults_offset = len(self.__spec.args) - len(self.__spec.defaults)
        return lambda: self.__spec.defaults[arg_idx - defaults_offset]


    def __kwoarg_view(self, arg_name):
        if arg_name in self.__kwargs:
            return lambda: self.__kwargs[arg_name]

        return lambda: self.__spec.kwonlydefaults[arg_name]


    def __varg_view(self, idx):
        return lambda: self.__args[idx]


    def __str__(self):
        return str(self.__dict__.keys())


    def __repr__(self):
        return repr(self.__dict__.keys())

In [16]:
def yoink(func):
    spec = getfullargspec(func)
    def wrapper(*args, **kwargs):
        av = ArgsView(spec, args, kwargs)
        return av
    return wrapper

In [17]:
h = yoink(f)

In [18]:
av = h(1,2)
av

dict_keys(['_ArgsView__spec', '_ArgsView__args', '_ArgsView__kwargs', 'x', 'y', 'a', '_vargs'])

In [19]:
av.x()

1

In [20]:
av.y()

2

In [21]:
li = [1, 2]
av._replace('x', li)

In [22]:
av.x()

[1, 2]

In [23]:
li[1] = 3

In [24]:
av.x()

[1, 3]

In [25]:
from copy import deepcopy
av2 = deepcopy(av)

In [26]:
av2

dict_keys(['_ArgsView__spec', '_ArgsView__args', '_ArgsView__kwargs', 'x', 'y', 'a', '_vargs'])

In [27]:
av2._replace('y', 100000)

In [28]:
av2.y()

100000

In [29]:
av.y()

2

In [30]:
h = yoink(g)

In [31]:
av = h(1,2,3,4,5,a=6)
av

dict_keys(['_ArgsView__spec', '_ArgsView__args', '_ArgsView__kwargs', 'x', 'y', '_vargs', 'a'])

In [32]:
av.a()

6

In [33]:
getattr(av, "a")()

6

In [34]:
av.x()

1

In [35]:
av.y()

2

In [36]:
av._vargs

[<function __main__.ArgsView.__varg_view.<locals>.<lambda>()>,
 <function __main__.ArgsView.__varg_view.<locals>.<lambda>()>,
 <function __main__.ArgsView.__varg_view.<locals>.<lambda>()>]

In [37]:
av._vargs[0]()

3

In [38]:
av._spec.kwonlyargs

['a']

In [39]:
arg_list = []
for arg in av._spec.args:
    arg_list.append(
        getattr(av, arg)()
    )

for argv in av._vargs:
    arg_list.append(
        argv()
    )

kwargs = dict()
for kw in av._spec.kwonlyargs:
    kwargs[kw] = getattr(av, kw)()

In [40]:
arg_list

[1, 2, 3, 4, 5]

In [41]:
kwargs

{'a': 6}

In [42]:
g(*arg_list, **kwargs)

8

In [43]:
def f(x):
    return lambda: x[1]

In [44]:
z = [1,2,3]

In [45]:
b=f(z)

In [46]:
b()

2

In [47]:
z[1]=100

In [48]:
b()

100